# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA4D_15/NACA4D_15.csv")

In [4]:
# printing the first 10 rows of the dataset
df.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
0,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-10.0,0.1,100000,-1.140,0.02494,0.005
1,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-9.0,0.1,100000,-1.027,0.02116,0.004
2,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-8.0,0.1,100000,-0.914,0.01697,0.004
3,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-7.0,0.1,100000,-0.800,0.01364,0.003
4,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-6.0,0.1,100000,-0.687,0.02483,0.003
5,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-5.0,0.1,100000,-0.572,0.01565,0.002
6,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-4.0,0.1,100000,-0.458,0.01114,0.002
7,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-3.0,0.1,100000,-0.344,0.00882,0.001
8,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-2.0,0.1,100000,-0.229,0.01391,0.001
9,5,0,5,0.006928,0.013123,0.018277,0.02209,0.024345,0.024985,0.024134,...,-0.008375,-0.005206,-0.002698,-0.001083,-1.0,0.1,100000,-0.115,0.01357,0.000


In [5]:
# checking for missing values in the dataset
df.isnull().sum()

t        0
m        0
p        0
yU1      0
yU2      0
yU3      0
yU4      0
yU5      0
yU6      0
yU7      0
yU8      0
yU9      0
yU10     0
yU11     0
yU12     0
yU13     0
yU14     0
yU15     0
yL1      0
yL2      0
yL3      0
yL4      0
yL5      0
yL6      0
yL7      0
yL8      0
yL9      0
yL10     0
yL11     0
yL12     0
yL13     0
yL14     0
yL15     0
alpha    0
M        0
Re       0
CL       0
CD       0
Cm       0
dtype: int64

In [6]:
# description of the dataset
df.describe()

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
count,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,...,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.00000,176400.000000,176400.000000,176400.000000,176400.000000
mean,20.000000,4.500000,40.000000,0.043185,0.072378,0.098342,0.119310,0.133160,0.139232,0.137718,...,-0.011315,-0.006839,-0.004054,-0.002540,0.000000,0.20000,300000.000000,0.692283,0.026646,-0.134151
std,10.000028,2.872289,22.912943,0.025991,0.035433,0.043725,0.050474,0.054749,0.056332,0.055247,...,0.023467,0.014910,0.007456,0.002464,6.055318,0.08165,141421.757093,0.926925,0.010960,0.113007
min,5.000000,0.000000,5.000000,0.006928,0.013123,0.018277,0.022090,0.024345,0.024985,0.024134,...,-0.058622,-0.036442,-0.018887,-0.007582,-10.000000,0.10000,100000.000000,-1.489000,0.001800,-0.514000
25%,10.000000,2.000000,22.500000,0.021753,0.042494,0.062212,0.078163,0.090537,0.095811,0.096468,...,-0.028432,-0.017678,-0.009537,-0.004367,-5.000000,0.10000,200000.000000,-0.011000,0.018990,-0.199000
50%,20.000000,4.500000,40.000000,0.039573,0.070644,0.098346,0.119243,0.132816,0.139388,0.137658,...,-0.012087,-0.007344,-0.004333,-0.002513,-0.000000,0.20000,300000.000000,0.691000,0.025390,-0.106000
75%,30.000000,7.000000,57.500000,0.059641,0.098268,0.132334,0.158673,0.176114,0.183007,0.179141,...,0.004018,0.002811,0.000762,-0.000748,5.000000,0.30000,400000.000000,1.379000,0.032540,-0.046000
max,35.000000,9.000000,75.000000,0.136063,0.169164,0.211244,0.240167,0.258280,0.264584,0.259470,...,0.065705,0.044603,0.022060,0.005317,10.000000,0.30000,500000.000000,3.599000,0.235470,0.025000


In [7]:
# shuffling all the rows
df = df.sample(frac=1)
df.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
155765,25,8,65,0.050911,0.086444,0.119564,0.148006,0.169775,0.183511,0.188638,...,0.012211,0.008443,0.003129,-0.001149,-2.0,0.2,300000,1.265,0.02986,-0.356
18891,20,1,5,0.040134,0.062784,0.082378,0.097873,0.106895,0.109212,0.105295,...,-0.030598,-0.019080,-0.009971,-0.004107,2.0,0.3,500000,0.393,0.01995,-0.013
157090,10,8,75,0.018221,0.036312,0.055317,0.073691,0.089894,0.102669,0.111190,...,0.049158,0.033760,0.016436,0.003352,-0.0,0.3,100000,1.618,0.03132,-0.433
100055,15,5,55,0.026538,0.049527,0.071618,0.091037,0.106097,0.115544,0.118738,...,0.002362,0.001517,0.000091,-0.001111,1.0,0.1,500000,0.902,0.01880,-0.184
145577,5,8,25,0.015397,0.038251,0.065067,0.089145,0.103230,0.104478,0.100854,...,0.019741,0.011699,0.005173,0.000926,-5.0,0.1,300000,0.287,0.01625,-0.162
151380,25,8,45,0.056957,0.094324,0.129868,0.160650,0.183891,0.197373,0.199597,...,-0.003786,-0.002572,-0.002339,-0.002454,2.0,0.2,400000,1.526,0.03135,-0.254
29758,20,1,55,0.029335,0.054905,0.076706,0.093421,0.104004,0.108023,0.105763,...,-0.027860,-0.017275,-0.009070,-0.003855,-9.0,0.2,300000,-1.009,0.02618,-0.024
79517,5,4,45,0.009106,0.020083,0.032244,0.044140,0.054207,0.061052,0.063667,...,0.010067,0.006092,0.002633,0.000293,1.0,0.2,200000,0.631,0.01365,-0.121
127641,35,7,15,0.107316,0.154084,0.193819,0.220978,0.237337,0.243037,0.235756,...,-0.035070,-0.022054,-0.011969,-0.005559,-7.0,0.2,400000,-0.016,0.02119,-0.087
123994,10,7,5,0.059430,0.092443,0.105787,0.113120,0.116310,0.114930,0.108982,...,0.003247,0.001509,0.000129,-0.000748,-0.0,0.3,500000,0.759,0.02041,-0.083


In [8]:
# dropping columns t, m & p
df = df.drop(columns=['t', 'm', 'p'], axis=1)
df.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL12,yL13,yL14,yL15,alpha,M,Re,CL,CD,Cm
155765,0.050911,0.086444,0.119564,0.148006,0.169775,0.183511,0.188638,0.185372,0.174630,0.157700,...,0.012211,0.008443,0.003129,-0.001149,-2.0,0.2,300000,1.265,0.02986,-0.356
18891,0.040134,0.062784,0.082378,0.097873,0.106895,0.109212,0.105295,0.096208,0.083330,0.068156,...,-0.030598,-0.019080,-0.009971,-0.004107,2.0,0.3,500000,0.393,0.01995,-0.013
157090,0.018221,0.036312,0.055317,0.073691,0.089894,0.102669,0.111190,0.115158,0.114775,0.110645,...,0.049158,0.033760,0.016436,0.003352,-0.0,0.3,100000,1.618,0.03132,-0.433
100055,0.026538,0.049527,0.071618,0.091037,0.106097,0.115544,0.118738,0.115673,0.106845,0.092344,...,0.002362,0.001517,0.000091,-0.001111,1.0,0.1,500000,0.902,0.01880,-0.184
145577,0.015397,0.038251,0.065067,0.089145,0.103230,0.104478,0.100854,0.093263,0.082071,0.068092,...,0.019741,0.011699,0.005173,0.000926,-5.0,0.1,300000,0.287,0.01625,-0.162
151380,0.056957,0.094324,0.129868,0.160650,0.183891,0.197373,0.199597,0.190182,0.171406,0.145383,...,-0.003786,-0.002572,-0.002339,-0.002454,2.0,0.2,400000,1.526,0.03135,-0.254
29758,0.029335,0.054905,0.076706,0.093421,0.104004,0.108023,0.105763,0.098114,0.086303,0.071583,...,-0.027860,-0.017275,-0.009070,-0.003855,-9.0,0.2,300000,-1.009,0.02618,-0.024
79517,0.009106,0.020083,0.032244,0.044140,0.054207,0.061052,0.063667,0.061734,0.056241,0.047956,...,0.010067,0.006092,0.002633,0.000293,1.0,0.2,200000,0.631,0.01365,-0.121
127641,0.107316,0.154084,0.193819,0.220978,0.237337,0.243037,0.235756,0.217373,0.190274,0.157261,...,-0.035070,-0.022054,-0.011969,-0.005559,-7.0,0.2,400000,-0.016,0.02119,-0.087
123994,0.059430,0.092443,0.105787,0.113120,0.116310,0.114930,0.108982,0.098918,0.085556,0.069950,...,0.003247,0.001509,0.000129,-0.000748,-0.0,0.3,500000,0.759,0.02041,-0.083


In [9]:
# defining the feature and target columns
X = df.drop(columns=['CL', 'CD', 'Cm'])
y = df[['CL', 'CD', 'Cm']]

In [10]:
# displaying the features table
X.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL9,yL10,yL11,yL12,yL13,yL14,yL15,alpha,M,Re
155765,0.050911,0.086444,0.119564,0.148006,0.169775,0.183511,0.188638,0.185372,0.174630,0.157700,...,-0.016378,0.001544,0.011040,0.012211,0.008443,0.003129,-0.001149,-2.0,0.2,300000
18891,0.040134,0.062784,0.082378,0.097873,0.106895,0.109212,0.105295,0.096208,0.083330,0.068156,...,-0.069695,-0.056968,-0.043566,-0.030598,-0.019080,-0.009971,-0.004107,2.0,0.3,500000
157090,0.018221,0.036312,0.055317,0.073691,0.089894,0.102669,0.111190,0.115158,0.114775,0.110645,...,0.038381,0.048259,0.055038,0.049158,0.033760,0.016436,0.003352,-0.0,0.3,100000
100055,0.026538,0.049527,0.071618,0.091037,0.106097,0.115544,0.118738,0.115673,0.106845,0.092344,...,-0.007737,-0.001518,0.001657,0.002362,0.001517,0.000091,-0.001111,1.0,0.1,500000
145577,0.015397,0.038251,0.065067,0.089145,0.103230,0.104478,0.100854,0.093263,0.082071,0.068092,...,0.043681,0.036618,0.028366,0.019741,0.011699,0.005173,0.000926,-5.0,0.1,300000
151380,0.056957,0.094324,0.129868,0.160650,0.183891,0.197373,0.199597,0.190182,0.171406,0.145383,...,-0.020345,-0.012173,-0.006750,-0.003786,-0.002572,-0.002339,-0.002454,2.0,0.2,400000
29758,0.029335,0.054905,0.076706,0.093421,0.104004,0.108023,0.105763,0.098114,0.086303,0.071583,...,-0.066438,-0.053292,-0.040125,-0.027860,-0.017275,-0.009070,-0.003855,-9.0,0.2,300000
79517,0.009106,0.020083,0.032244,0.044140,0.054207,0.061052,0.063667,0.061734,0.056241,0.047956,...,0.018022,0.016672,0.013833,0.010067,0.006092,0.002633,0.000293,1.0,0.2,200000
127641,0.107316,0.154084,0.193819,0.220978,0.237337,0.243037,0.235756,0.217373,0.190274,0.157261,...,-0.082208,-0.066298,-0.050152,-0.035070,-0.022054,-0.011969,-0.005559,-7.0,0.2,400000
123994,0.059430,0.092443,0.105787,0.113120,0.116310,0.114930,0.108982,0.098918,0.085556,0.069950,...,0.008661,0.006982,0.005140,0.003247,0.001509,0.000129,-0.000748,-0.0,0.3,500000


In [11]:
# displaying the targets table
y.head(10)

,CL,CD,Cm
155765,1.265,0.02986,-0.356
18891,0.393,0.01995,-0.013
157090,1.618,0.03132,-0.433
100055,0.902,0.01880,-0.184
145577,0.287,0.01625,-0.162
151380,1.526,0.03135,-0.254
29758,-1.009,0.02618,-0.024
79517,0.631,0.01365,-0.121
127641,-0.016,0.02119,-0.087
123994,0.759,0.02041,-0.083


## Splitting the Dataset

In [12]:
# splitting dataset into training, validation & testing dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [14]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.218736,0.323761,0.407181,0.482418,0.574298,0.685286,0.819846,0.983815,1.188213,1.457598,...,0.388945,0.725153,1.203529,1.569719,1.727458,1.690243,1.048785,-0.990537,-1.223895,0.705621
1,-0.062131,0.179605,0.258907,0.261990,0.235613,0.189295,0.127838,0.053846,-0.030650,-0.118548,...,-1.682830,-1.698881,-1.687390,-1.658998,-1.636035,-1.627135,-1.606590,1.487688,-1.223895,-0.709311
2,0.505608,0.778526,0.882564,0.906370,0.905706,0.890828,0.867019,0.835382,0.798830,0.765592,...,-1.668275,-1.581980,-1.482847,-1.397433,-1.343445,-1.338975,-1.446580,0.496398,0.001359,-0.001845
3,0.015430,0.122189,0.173372,0.196941,0.215717,0.227115,0.225968,0.205029,0.167879,0.115516,...,-0.650038,-0.665142,-0.679013,-0.682905,-0.681993,-0.682340,-0.672920,-0.660107,-1.223895,0.705621
4,0.335859,0.499708,0.561020,0.571247,0.564630,0.538806,0.488659,0.424909,0.348616,0.260389,...,-1.299004,-1.312036,-1.307784,-1.290467,-1.277062,-1.279088,-1.298252,0.000753,0.001359,0.705621


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,3.178399,2.360676,2.024129,2.014545,2.006959,1.961890,1.900141,1.819236,1.708879,1.567187,...,-0.930130,-0.966887,-0.988898,-0.998075,-1.011644,-1.056243,-1.221645,0.826828,1.226614,-1.416777
1,-1.018836,-1.114001,-1.130602,-1.108901,-1.065029,-1.008984,-0.947160,-0.882804,-0.818480,-0.769672,...,0.845626,0.817489,0.754841,0.698227,0.671632,0.689252,0.836028,1.322473,0.001359,-1.416777
2,-0.641698,-0.761399,-0.962141,-1.098508,-1.189723,-1.254731,-1.304867,-1.345516,-1.376960,-1.394210,...,0.274226,0.174016,0.077211,0.011410,-0.011422,0.032002,0.318442,0.000753,-1.223895,1.413087
3,-1.394495,-1.671591,-1.830081,-1.924770,-1.986019,-2.026638,-2.054592,-2.071819,-2.074797,-2.055060,...,0.398951,0.293979,0.191928,0.124410,0.108577,0.180827,0.590408,0.992043,0.001359,0.705621
4,-0.433220,-0.392364,-0.377807,-0.375612,-0.374101,-0.376873,-0.388500,-0.413178,-0.446593,-0.484457,...,-0.441119,-0.481948,-0.519132,-0.539136,-0.542563,-0.524879,-0.413358,-0.329677,-1.223895,-0.709311


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,-0.062131,0.179605,0.258907,0.261990,0.235613,0.189295,0.127838,0.053846,-0.030650,-0.118548,...,-1.682830,-1.698881,-1.687390,-1.658998,-1.636035,-1.627135,-1.606590,1.487688,-1.223895,1.413087
1,0.916331,0.963297,0.992753,0.992375,0.957518,0.894825,0.831084,0.760603,0.677212,0.578312,...,-0.980607,-1.004982,-1.017234,-1.016647,-1.016767,-1.032285,-1.088237,0.826828,0.001359,0.705621
2,-1.382458,-1.642213,-1.780704,-1.854523,-1.893702,-1.910674,-1.912645,-1.900601,-1.869661,-1.808603,...,0.607892,0.547666,0.502284,0.476817,0.483967,0.557716,0.896260,1.157258,-1.223895,-0.709311
3,0.417718,0.546843,0.685280,0.776728,0.786324,0.714149,0.652946,0.598014,0.535679,0.457705,...,0.532115,0.451005,0.357420,0.279995,0.227393,0.189942,0.143524,0.661613,1.226614,0.705621
4,0.732071,0.786207,0.850558,0.886932,0.871378,0.804578,0.743323,0.681907,0.610326,0.523111,...,-0.222835,-0.274625,-0.326672,-0.364284,-0.389736,-0.414756,-0.461473,0.331183,1.226614,-0.709311


In [15]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
157987,0.947,0.02764,-0.413
8230,1.243,0.03568,-0.019
50560,0.865,0.03334,-0.100
63069,-0.067,0.02276,-0.086
43648,0.311,0.02627,-0.051


,CL,CD,Cm
143340,1.872,0.02229,-0.085
82023,1.564,0.02886,-0.153
35689,0.212,0.00939,-0.022
11209,0.697,0.01636,-0.003
45074,0.040,0.02027,-0.059


,CL,CD,Cm
8293,1.243,0.03089,-0.019
76728,1.288,0.03228,-0.087
33113,0.994,0.01342,-0.056
146657,1.610,0.03061,-0.170
111753,1.132,0.03123,-0.126


In [16]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

## Network Performance Due to Multiple Hidden Layers

### Case 1: Model [64, 3]

In [17]:
# RMSE and R² metric scores on test data for Model M_64
M_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64...")

    M_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_df = pd.DataFrame(M_64_metrics)

print(results_M_64_df.mean())



Run 1/20 for Model M_64...


I0000 00:00:1764175345.039831 1934507 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764175347.459075 1935800 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_64...

Run 3/20 for Model M_64...

Run 4/20 for Model M_64...

Run 5/20 for Model M_64...

Run 6/20 for Model M_64...

Run 7/20 for Model M_64...

Run 8/20 for Model M_64...

Run 9/20 for Model M_64...

Run 10/20 for Model M_64...

Run 11/20 for Model M_64...

Run 12/20 for Model M_64...

Run 13/20 for Model M_64...

Run 14/20 for Model M_64...

Run 15/20 for Model M_64...

Run 16/20 for Model M_64...

Run 17/20 for Model M_64...

Run 18/20 for Model M_64...

Run 19/20 for Model M_64...

Run 20/20 for Model M_64...
CL_rmse    0.006077
CD_rmse    0.004670
Cm_rmse    0.002374
CL_r2      0.999956
CD_r2      0.821334
Cm_r2      0.999555
dtype: float32


### Case 2: Model [64, 32, 3]

In [18]:
# RMSE and R² metric scores on test data for Model M_64_32
M_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32...")
    
    M_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_df = pd.DataFrame(M_64_32_metrics)

print(results_M_64_32_df.mean())



Run 1/20 for Model M_64_32...

Run 2/20 for Model M_64_32...

Run 3/20 for Model M_64_32...

Run 4/20 for Model M_64_32...

Run 5/20 for Model M_64_32...

Run 6/20 for Model M_64_32...

Run 7/20 for Model M_64_32...

Run 8/20 for Model M_64_32...

Run 9/20 for Model M_64_32...

Run 10/20 for Model M_64_32...

Run 11/20 for Model M_64_32...

Run 12/20 for Model M_64_32...

Run 13/20 for Model M_64_32...

Run 14/20 for Model M_64_32...

Run 15/20 for Model M_64_32...

Run 16/20 for Model M_64_32...

Run 17/20 for Model M_64_32...

Run 18/20 for Model M_64_32...

Run 19/20 for Model M_64_32...

Run 20/20 for Model M_64_32...
CL_rmse    0.004789
CD_rmse    0.004284
Cm_rmse    0.002098
CL_r2      0.999973
CD_r2      0.850207
Cm_r2      0.999651
dtype: float32


### Case 3: Model [64, 32, 16, 3]

In [19]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")
    
    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.004540
CD_rmse    0.004256
Cm_rmse    0.001997
CL_r2      0.999976
CD_r2      0.852136
Cm_r2      0.999686
dtype: float32


### Case 4: Model [64, 32, 16, 8, 3]

In [20]:
# RMSE and R² metric scores on test data for Model M_64_32_16_8
M_64_32_16_8_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16_8...")
    
    M_64_32_16_8 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16_8.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16_8.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16_8.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_8_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_8_df = pd.DataFrame(M_64_32_16_8_metrics)

print(results_M_64_32_16_8_df.mean())



Run 1/20 for Model M_64_32_16_8...

Run 2/20 for Model M_64_32_16_8...

Run 3/20 for Model M_64_32_16_8...

Run 4/20 for Model M_64_32_16_8...

Run 5/20 for Model M_64_32_16_8...

Run 6/20 for Model M_64_32_16_8...

Run 7/20 for Model M_64_32_16_8...

Run 8/20 for Model M_64_32_16_8...

Run 9/20 for Model M_64_32_16_8...

Run 10/20 for Model M_64_32_16_8...

Run 11/20 for Model M_64_32_16_8...

Run 12/20 for Model M_64_32_16_8...

Run 13/20 for Model M_64_32_16_8...

Run 14/20 for Model M_64_32_16_8...

Run 15/20 for Model M_64_32_16_8...

Run 16/20 for Model M_64_32_16_8...

Run 17/20 for Model M_64_32_16_8...

Run 18/20 for Model M_64_32_16_8...

Run 19/20 for Model M_64_32_16_8...

Run 20/20 for Model M_64_32_16_8...
CL_rmse    0.004478
CD_rmse    0.004403
Cm_rmse    0.002031
CL_r2      0.999976
CD_r2      0.839429
Cm_r2      0.999675
dtype: float32


In [21]:
M_64_stats = results_M_64_df.mean()
M_64_32_stats = results_M_64_32_df.mean()
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_64_32_16_8_stats = results_M_64_32_16_8_df.mean()

architectures = [
    '64, 3',
    '64, 32, 3',
    '64, 32, 16, 3',
    '64, 32, 16, 8, 3'
]

table_data = [
    (M_64_stats['CL_rmse'], M_64_stats['CD_rmse'], M_64_stats['Cm_rmse'],
     M_64_stats['CL_r2'], M_64_stats['CD_r2'], M_64_stats['Cm_r2']),
    
    (M_64_32_stats['CL_rmse'], M_64_32_stats['CD_rmse'], M_64_32_stats['Cm_rmse'],
     M_64_32_stats['CL_r2'], M_64_32_stats['CD_r2'], M_64_32_stats['Cm_r2']),
    
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_64_32_16_8_stats['CL_rmse'], M_64_32_16_8_stats['CD_rmse'], M_64_32_16_8_stats['Cm_rmse'],
     M_64_32_16_8_stats['CL_r2'], M_64_32_16_8_stats['CD_r2'], M_64_32_16_8_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 5), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))


## Network Performance Due to Increasing Neurons

### Case 1: Model [64, 32, 16, 3]

In [22]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")

    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.004882
CD_rmse    0.004210
Cm_rmse    0.002079
CL_r2      0.999972
CD_r2      0.855346
Cm_r2      0.999658
dtype: float32


### Case 2: Model [128, 64, 32, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_128_64_32
M_128_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_128_64_32...")

    M_128_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_128_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_128_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_128_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_128_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_128_64_32_df = pd.DataFrame(M_128_64_32_metrics)

print(results_M_128_64_32_df.mean())



Run 1/20 for Model M_128_64_32...

Run 2/20 for Model M_128_64_32...

Run 3/20 for Model M_128_64_32...

Run 4/20 for Model M_128_64_32...

Run 5/20 for Model M_128_64_32...

Run 6/20 for Model M_128_64_32...

Run 7/20 for Model M_128_64_32...

Run 8/20 for Model M_128_64_32...

Run 9/20 for Model M_128_64_32...

Run 10/20 for Model M_128_64_32...

Run 11/20 for Model M_128_64_32...

Run 12/20 for Model M_128_64_32...

Run 13/20 for Model M_128_64_32...

Run 14/20 for Model M_128_64_32...

Run 15/20 for Model M_128_64_32...

Run 16/20 for Model M_128_64_32...

Run 17/20 for Model M_128_64_32...

Run 18/20 for Model M_128_64_32...

Run 19/20 for Model M_128_64_32...

Run 20/20 for Model M_128_64_32...
CL_rmse    0.003589
CD_rmse    0.003887
Cm_rmse    0.001435
CL_r2      0.999985
CD_r2      0.876686
Cm_r2      0.999837
dtype: float32


### Case 3: Model [256, 128, 64, 3]

In [24]:
# RMSE and R² metric scores on test data for Model M_256_128_64
M_256_128_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_256_128_64...")

    M_256_128_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_256_128_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_256_128_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_256_128_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_256_128_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_256_128_64_df = pd.DataFrame(M_256_128_64_metrics)

print(results_M_256_128_64_df.mean())



Run 1/20 for Model M_256_128_64...

Run 2/20 for Model M_256_128_64...

Run 3/20 for Model M_256_128_64...

Run 4/20 for Model M_256_128_64...

Run 5/20 for Model M_256_128_64...

Run 6/20 for Model M_256_128_64...

Run 7/20 for Model M_256_128_64...

Run 8/20 for Model M_256_128_64...

Run 9/20 for Model M_256_128_64...

Run 10/20 for Model M_256_128_64...

Run 11/20 for Model M_256_128_64...

Run 12/20 for Model M_256_128_64...

Run 13/20 for Model M_256_128_64...

Run 14/20 for Model M_256_128_64...

Run 15/20 for Model M_256_128_64...

Run 16/20 for Model M_256_128_64...

Run 17/20 for Model M_256_128_64...

Run 18/20 for Model M_256_128_64...

Run 19/20 for Model M_256_128_64...

Run 20/20 for Model M_256_128_64...
CL_rmse    0.003168
CD_rmse    0.003581
Cm_rmse    0.001131
CL_r2      0.999987
CD_r2      0.895340
Cm_r2      0.999898
dtype: float32


### Case 4: Model [512, 256, 128, 3]

In [25]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...

Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.002750
CD_rmse    0.003425
Cm_rmse    0.000968
CL_r2      0.999990
CD_r2      0.904181
Cm_r2      0.999922
dtype: float32


### Case 5: Model [1024, 512, 256, 3]

In [26]:
# RMSE and R² metric scores on test data for Model M_1024_512_256
M_1024_512_256_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_1024_512_256...")

    M_1024_512_256 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_1024_512_256.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_1024_512_256.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_1024_512_256.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_1024_512_256_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_1024_512_256_df = pd.DataFrame(M_1024_512_256_metrics)

print(results_M_1024_512_256_df.mean())



Run 1/20 for Model M_1024_512_256...

Run 2/20 for Model M_1024_512_256...

Run 3/20 for Model M_1024_512_256...

Run 4/20 for Model M_1024_512_256...

Run 5/20 for Model M_1024_512_256...

Run 6/20 for Model M_1024_512_256...

Run 7/20 for Model M_1024_512_256...

Run 8/20 for Model M_1024_512_256...

Run 9/20 for Model M_1024_512_256...

Run 10/20 for Model M_1024_512_256...

Run 11/20 for Model M_1024_512_256...

Run 12/20 for Model M_1024_512_256...

Run 13/20 for Model M_1024_512_256...

Run 14/20 for Model M_1024_512_256...

Run 15/20 for Model M_1024_512_256...

Run 16/20 for Model M_1024_512_256...

Run 17/20 for Model M_1024_512_256...

Run 18/20 for Model M_1024_512_256...

Run 19/20 for Model M_1024_512_256...

Run 20/20 for Model M_1024_512_256...
CL_rmse    0.002573
CD_rmse    0.003352
Cm_rmse    0.000895
CL_r2      0.999991
CD_r2      0.908223
Cm_r2      0.999935
dtype: float32


In [27]:
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_128_64_32_stats = results_M_128_64_32_df.mean()
M_256_128_64_stats = results_M_256_128_64_df.mean()
M_512_256_128_stats = results_M_512_256_128_df.mean()
M_1024_512_256_stats = results_M_1024_512_256_df.mean()

architectures = [
    '64, 32, 16, 3',
    '128, 64, 32, 3',
    '256, 128, 64, 3',
    '512, 256, 128, 3',
    '1024, 512, 256, 3'
]

table_data = [
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_128_64_32_stats['CL_rmse'], M_128_64_32_stats['CD_rmse'], M_128_64_32_stats['Cm_rmse'],
     M_128_64_32_stats['CL_r2'], M_128_64_32_stats['CD_r2'], M_128_64_32_stats['Cm_r2']),
    
    (M_256_128_64_stats['CL_rmse'], M_256_128_64_stats['CD_rmse'], M_256_128_64_stats['Cm_rmse'],
     M_256_128_64_stats['CL_r2'], M_256_128_64_stats['CD_r2'], M_256_128_64_stats['Cm_r2']),
    
    (M_512_256_128_stats['CL_rmse'], M_512_256_128_stats['CD_rmse'], M_512_256_128_stats['Cm_rmse'],
     M_512_256_128_stats['CL_r2'], M_512_256_128_stats['CD_r2'], M_512_256_128_stats['Cm_r2']),
    
    (M_1024_512_256_stats['CL_rmse'], M_1024_512_256_stats['CD_rmse'], M_1024_512_256_stats['Cm_rmse'],
     M_1024_512_256_stats['CL_r2'], M_1024_512_256_stats['CD_r2'], M_1024_512_256_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 6), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))
